In [1]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
 
#import itertools
import pandas as pd


import db_setup

from db_setup import Subject, Portfolio, ChoiceProblem, Selection, db, app
 

session = db.session

Create datafames from SQL tables.

In [2]:
subs_df = pd.read_sql(session.query(Subject).filter(Subject.passquiz==True).statement,session.bind) 
sels_df = pd.read_sql(session.query(Selection).statement,session.bind) 

In [3]:
## our CP_types

CP_type_list = [
['110000', 'init'],
['111000', 'cash_attr'],
['110100', 'long_attr'],
['110010', 'cash_comp'],
['110001', 'long_comp'],
]

#add a column with the ID
for t in CP_type_list:
    t.append(session.query(ChoiceProblem).filter(ChoiceProblem.CP_type == t[0]).one().id)

In [4]:
CP_type_list

[['110000', 'init', 1],
 ['111000', 'cash_attr', 2],
 ['110100', 'long_attr', 3],
 ['110010', 'cash_comp', 4],
 ['110001', 'long_comp', 5]]

In [5]:
# then construct the type of each subject (cash or long type)


def init_type(x):
    init_sel = session.query(Selection).filter(Selection.choice_problem == 1).filter(Selection.subject == x).one()

    return init_sel.choice #1 == cash, 2 == long

#add the to the dataframe
subs_df['init_type'] = subs_df['id'].apply(init_type)

In [12]:
#for each round, check consistincy 

#create new cols for each
for t in CP_type_list[1:]:
    
    subs_df["sw_"+t[1]] = 'cons'
    subs_df["sw_"+t[1]+'_direction'] = '0'

    
##switch for people who did swtiching
    
for sub in session.query(Subject).filter(Subject.passquiz==True).all():
    for t in CP_type_list[1:]:
        
        sel = session.query(Selection).filter(Selection.choice_problem == t[2]).filter(Selection.subject == sub.id).one()
        init_sel = session.query(Selection).filter(Selection.choice_problem == 1).filter(Selection.subject == sub.id).one()
        
        if sel.choice != init_sel.choice:
            subs_df.loc[(subs_df.id== sub.id),"sw_"+t[1]] = 'switch'
            if (t[2] in [2,4] and sel.choice in [1,3,5]) or (t[2] in [3,5] and sel.choice in [2,4,6]):
                subs_df.loc[(subs_df.id== sub.id),"sw_"+t[1]+'_direction'] = 'good'
            else:
                subs_df.loc[(subs_df.id== sub.id),"sw_"+t[1]+'_direction'] = 'bad'

In [13]:
subs_df

,id,idCode,hashed_id,time_start,tryquiz,passquiz,current_rnd,order,payment_question,total_payment,...,location,init_type,sw_cash_attr,sw_long_attr,sw_cash_comp,sw_long_comp,sw_cash_attr_direction,sw_long_attr_direction,sw_cash_comp_direction,sw_long_comp_direction
0,2,A1V2H0UF94ATWY,cee3a8f1,2018-12-17 18:54:35.737793,True,True,0,62,3,25,...,0,1,cons,cons,cons,cons,0,0,0,0
1,3,APGX2WZ59OWDN,8cb5b068,2018-12-17 19:14:01.343523,True,True,0,113,0,25,...,0,1,cons,cons,cons,cons,0,0,0,0
2,4,A9HQ3E0F2AGVO,f4c2e991,2018-12-17 19:37:43.151194,True,True,0,15,3,25,...,0,1,cons,cons,cons,cons,0,0,0,0
3,5,AR1IWBDA7MC86,63c8448f,2018-12-17 19:42:31.122325,True,True,0,51,1,25,...,0,1,cons,cons,cons,cons,0,0,0,0
4,6,evan1111111,ecb31af1,2018-12-18 15:14:04.453295,True,True,0,25,2,25,...,1,2,cons,switch,cons,cons,0,good,0,0


In [8]:
sels_df

,id,choice_problem,choice,time_start,time_end,beginning_price,ending_price,chart_time,click_hist,rnd,subject
0,1,3,1,2018-12-17 18:58:09.674500,2018-12-17 18:58:51.709570,24142,None,0,"[[""1"",""2018-12-17T18:58:32.620Z""]]",0,2
1,2,4,1,2018-12-17 18:58:51.750082,2018-12-17 18:59:10.638364,24142,None,0,"[[""1"",""2018-12-17T18:59:09.596Z""]]",1,2
2,3,2,1,2018-12-17 18:59:10.668609,2018-12-17 18:59:18.704569,24142,None,0,"[[""1"",""2018-12-17T18:59:16.045Z""]]",2,2
3,4,1,1,2018-12-17 18:59:18.775281,2018-12-17 18:59:24.924825,24142,None,0,"[[""1"",""2018-12-17T18:59:24.268Z""]]",3,2
4,5,5,1,2018-12-17 18:59:25.001513,2018-12-17 18:59:46.875647,24142,None,0,"[[""1"",""2018-12-17T18:59:33.564Z""]]",4,2
5,6,5,1,2018-12-17 19:15:33.284025,2018-12-17 19:15:48.067220,24142,None,4446,"[[""1"",""2018-12-17T19:16:18.923Z""]]",0,3
6,7,3,1,2018-12-17 19:15:48.127167,2018-12-17 19:15:57.267073,24142,None,2807,"[[""1"",""2018-12-17T19:16:27.787Z""]]",1,3
7,8,4,1,2018-12-17 19:15:57.300741,2018-12-17 19:16:05.556318,24142,None,1542,"[[""5"",""2018-12-17T19:16:35.227Z""],[""1"",""2018-1...",2,3
8,9,2,1,2018-12-17 19:16:05.588071,2018-12-17 19:16:09.527697,24142,None,0,"[[""1"",""2018-12-17T19:16:42.098Z""]]",3,3
9,10,1,1,2018-12-17 19:16:09.557043,2018-12-17 19:16:14.452586,24142,None,759,"[[""1"",""2018-12-17T19:16:46.890Z""]]",4,3
